<a href="https://colab.research.google.com/github/valfaro-v2/MagolegoTeamProject/blob/master/Shap_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shap
import transformers
import torch
import numpy as np
import scipy as sp
import torch

import pandas as pd

In [2]:
df_shap = pd.read_csv('tripadvisor_hotel_reviews.csv')

five_star_ratings_index = df_shap.index[df_shap['Rating']==5].tolist() #select rows labeled with 5

df_shap = df_shap.drop(index=five_star_ratings_index) #drop the rows that we selected
df_shap = df_shap.reset_index()
df_shap = df_shap.drop(columns=['index'])

NUMBER_OF_ROWS = df_shap.shape[0]

df_shap_sentiment = pd.DataFrame(data = np.zeros(NUMBER_OF_ROWS), columns=['Sentiment'],dtype='int64') #we create the sentiment column
df_shap_model = pd.concat([df_shap,df_shap_sentiment], axis = 1)

for i in range(NUMBER_OF_ROWS): #we update the sentiment column
  if df_shap_model.at[i,'Rating']==4:
    df_shap_model.at[i,'Sentiment']=1
    
print('The first rows of the dataframe would now look as follows:')
df_shap_model = df_shap_model.drop(columns=['Rating'])
df_shap_model

The first rows of the dataframe would now look as follows:


,Review,Sentiment
0,nice hotel expensive parking got good deal sta...,1
1,ok nothing special charge diamond member hilto...,0
2,nice rooms not 4* experience hotel monaco seat...,0
3,"excellent staff, housekeeping quality hotel ch...",1
4,"poor value stayed monaco seattle july, nice ho...",0
...,...,...
11432,not impressed unfriendly staff checked asked h...,0
11433,great location price view hotel great quick pl...,1
11434,"ok just looks nice modern outside, desk staff ...",0
11435,hotel theft ruined vacation hotel opened sept ...,0


In [6]:
short_data = [v[:500] for v in df_shap_model["Review"][:20]]

In [7]:
classifier = transformers.pipeline('sentiment-analysis', return_all_scores=True)
classifier(short_data[:2])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[[{'label': 'NEGATIVE', 'score': 0.9736725687980652},
  {'label': 'POSITIVE', 'score': 0.02632744610309601}],
 [{'label': 'NEGATIVE', 'score': 0.9972202777862549},
  {'label': 'POSITIVE', 'score': 0.0027797056827694178}]]

In [8]:
explainer = shap.Explainer(classifier)

In [9]:
shap_values = explainer(short_data[:2])

  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer:  50%|█████     | 1/2 [00:00<?, ?it/s]

  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 3it [03:12, 96.27s/it]


In [10]:
shap.plots.text(shap_values[:,:,"POSITIVE"])

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
